<a href="https://colab.research.google.com/github/angrybrd/licode/blob/master/LLMFineTuning_Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U transformers datasets bitsandbytes  trl peft  huggingface_hub

In [ ]:
from huggingface_hub import login
from google.colab import userdata
login(token=userdata.get('hftoken'))


In [ ]:
!pip install -U bitsandbytes

In [ ]:
!pip install accelerate

In [ ]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# model_name = "Cagatayd/llama3.2-1B-Instruct-Egitim"
# model_name = "unsloth/Llama-3.2-1B-Instruct"
model_name = "meta-llama/Llama-3.2-1B-Instruct"

config_8bit = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(


                                                  model_name,
                                                  quantization_config=config_8bit,
                                                  device_map= "auto",
                                                  trust_remote_code =True

                                                  )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model_8bit

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear8bitLt(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,padding_side="left",trust_remote_code= True)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("MattCoddity/dockerNLcommands")
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 2415
    })
})

In [ ]:
dataset['train'][1]

{'input': 'Fetch the containers that have exited with a status code of 1.',
 'output': "docker ps -a --filter 'status=exited' --filter 'exited=1'",
 'instruction': 'translate this sentence in docker command'}

In [ ]:
from datasets import DatasetDict

train_val_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

dataset = DatasetDict(

                      {'train':train_val_split['train'],
                       'validation':train_val_split['test']}

)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 483
    })
})

In [ ]:
def to_chat_template(example):


    messages = [

                {"role": 'system','content':example['instruction']},
                {"role": 'user','content':example['input']},
                {"role": 'assistant','content':example['output']}

                ]


    return {'text':messages}

dataset = dataset.map(to_chat_template)
dataset


DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 483
    })
})

In [ ]:
dataset['train']['text'][0]

[{'content': 'translate this sentence in docker command', 'role': 'system'},
 {'content': 'Find the repository, tag, and ID of the images that were created before the latest nginx image.',
  'role': 'user'},
 {'content': 'docker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"',
  'role': 'assistant'}]

In [ ]:
tokenizer.apply_chat_template(dataset['train']['text'][0],tokenize=False)

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 16 Nov 2025\n\ntranslate this sentence in docker command<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nFind the repository, tag, and ID of the images that were created before the latest nginx image.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\ndocker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"<|eot_id|>'

In [ ]:
def apply_chat_temp(example):

    new_text = tokenizer.apply_chat_template(example['text'],tokenize=False)

    return {'text':new_text}


dataset = dataset.map(apply_chat_temp)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input', 'output', 'instruction', 'text'],
        num_rows: 483
    })
})

In [ ]:
dataset['train']['text'][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 16 Nov 2025\n\ntranslate this sentence in docker command<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nFind the repository, tag, and ID of the images that were created before the latest nginx image.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\ndocker images -f "before=nginx:latest" --format "{{.Repository}},{{.Tag}},{{.ID}}"<|eot_id|>'

In [ ]:
def tokenize_fn(example):

    return tokenizer(example['text'])


tokenized_dataset = dataset.map(tokenize_fn,batched=True,remove_columns=['input', 'output', 'instruction', 'text'])
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 483
    })
})

In [ ]:
len(tokenized_dataset['train']['input_ids'][1])

79

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False,return_tensors="pt")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

'<|eot_id|>'

In [ ]:
from torch.utils.data import DataLoader


data_loader = DataLoader(
    tokenized_dataset['train'],
    batch_size=2,
    collate_fn=data_collator
)
for step, batch in enumerate(data_loader):
    print(f"Batch {step}")
    print("input_ids.shape:", batch["input_ids"].shape)

    print("attention_mask.shape:", batch["attention_mask"].shape)


    if step >= 3:
        break

Batch 0
input_ids.shape: torch.Size([2, 88])
attention_mask.shape: torch.Size([2, 88])
Batch 1
input_ids.shape: torch.Size([2, 79])
attention_mask.shape: torch.Size([2, 79])
Batch 2
input_ids.shape: torch.Size([2, 80])
attention_mask.shape: torch.Size([2, 80])
Batch 3
input_ids.shape: torch.Size([2, 78])
attention_mask.shape: torch.Size([2, 78])


In [ ]:
from peft import  LoraConfig, get_peft_model
import copy

model_8bit_clone = copy.deepcopy(model_8bit)


lora_config = LoraConfig(

                         r= 64,
                         lora_alpha = 32,
                         target_modules = ["q_proj","k_proj","v_proj","o_proj",
                      "gate_proj","up_proj","down_proj"],
                         lora_dropout = 0.05,
                         bias = "none",
                         task_type = "CAUSAL_LM"

)

model_8bit_lora = get_peft_model(model_8bit_clone,lora_config)
model_8bit_lora.print_trainable_parameters()

trainable params: 45,088,768 || all params: 1,280,903,168 || trainable%: 3.5201


In [ ]:
model_8bit_lora

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer


training_args = TrainingArguments(

                                  output_dir = "./resutls",
                                  per_device_train_batch_size = 2,
                                  per_device_eval_batch_size = 2,
                                  eval_steps = 10,
                                  eval_strategy = "steps",
                                  save_steps = 20,
                                  save_strategy = "steps",
                                  # num_train_epochs = 1,
                                  max_steps = 60,
                                  learning_rate = 3e-5,
                                  weight_decay = 0.01,
                                  warmup_steps = 5,
                                  fp16 = True,
                                  gradient_accumulation_steps = 4,
                                  optim = "adamw_torch",
                                  logging_steps = 10,
                                  report_to = "none"


                                  )

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1932
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 483
    })
})

In [ ]:
trainer = SFTTrainer(

                     model =model_8bit_lora,
                     train_dataset =tokenized_dataset['train'],
                     eval_dataset = tokenized_dataset['validation'],
                     args = training_args,
                     data_collator= data_collator,
                    #  tokenizer = tokenizer
                    processing_class = tokenizer, # in tne new version
)

trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
10,4.450000,3.275873,2.252475,5693.000000,0.530935
20,2.709400,2.124186,2.140661,11434.000000,0.631342
30,1.890000,1.617373,1.717732,17167.000000,0.736946
40,1.434000,1.299356,1.304851,22732.000000,0.776687
50,1.272800,1.192893,1.196732,28531.000000,0.802616
60,1.189800,1.166990,1.170796,34186.000000,0.804166


TrainOutput(global_step=60, training_loss=2.1576615969340005, metrics={'train_runtime': 313.6397, 'train_samples_per_second': 1.53, 'train_steps_per_second': 0.191, 'total_flos': 226414707916800.0, 'train_loss': 2.1576615969340005, 'epoch': 0.2484472049689441})

In [ ]:
model_8bit_lora

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [ ]:
import torch

torch.set_printoptions(precision=8, sci_mode=False)

layer0 = model_8bit_lora.model.model.layers[0]

model_8bit_weights = layer0.self_attn.q_proj.weight
print("model_8bit_weights:", model_8bit_weights)

model_8bit_weights: Parameter containing:
Parameter(Int8Params([[-44,  16,  61,  ..., -22, -29,  50],
            [ 14,  70,  65,  ..., -39, -18,  13],
            [ 16,  14,  30,  ..., -34, -34, -24],
            ...,
            [ 17,  20,  40,  ..., -40, -15, -16],
            [ 32, -35,  50,  ..., -17, -41, -21],
            [-14, -30,  -7,  ...,  30,   5,  -2]], device='cuda:0',
           dtype=torch.int8))


In [ ]:
import torch

torch.set_printoptions(precision=8, sci_mode=False)

layer0 = model_8bit_lora.model.model.layers[0]

# LoRA A matrisi
weight_A = layer0.self_attn.q_proj.lora_A["default"].weight
print("LoRA A weight:", weight_A)

LoRA A weight: Parameter containing:
tensor([[ 0.00980067, -0.02102597, -0.00936873,  ..., -0.01195977,
         -0.01148480,  0.01726137],
        [-0.01799890, -0.00860392, -0.01328950,  ...,  0.00568264,
         -0.01869317,  0.02176888],
        [ 0.01471585, -0.01597361,  0.00469105,  ..., -0.01293239,
          0.00667298,  0.00690165],
        ...,
        [-0.00339139, -0.01102571, -0.01443482,  ..., -0.01194018,
         -0.01464228,  0.00110960],
        [ 0.01806163, -0.00026490,  0.00095074,  ..., -0.01831059,
         -0.01979986,  0.01417157],
        [-0.00707271, -0.00113285,  0.01433204,  ..., -0.01112448,
          0.00639438,  0.00755331]], device='cuda:0', requires_grad=True)


In [ ]:
lora_path = "./lora_adapters"
model_8bit_lora.save_pretrained(lora_path)

In [ ]:

base_model = AutoModelForCausalLM.from_pretrained(


                                                  model_name,

                                                  device_map= "auto",
                                                  trust_remote_code =True

                                                  )

In [ ]:
from peft import PeftModel

base_model =PeftModel.from_pretrained(base_model,lora_path)
base_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [ ]:
base_model = base_model.merge_and_unload()
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (ro

In [ ]:
#base_model.push_to_hub("rmajumder/Llama3.2-doker-egitim")
#tokenizer.push_to_hub("rmajumder/Llama3.2-doker-egitim")

In [ ]:
dataset['train'][1]

{'input': 'Please show me the Docker containers that have exited and are related to the mongo image.',
 'output': "docker ps -a --filter 'status=exited' --filter 'ancestor=mongo'",
 'instruction': 'translate this sentence in docker command',
 'text': "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 16 Nov 2025\n\ntranslate this sentence in docker command<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPlease show me the Docker containers that have exited and are related to the mongo image.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\ndocker ps -a --filter 'status=exited' --filter 'ancestor=mongo'<|eot_id|>"}

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=base_model,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    tokenizer = tokenizer
)
messages = [
    {"role": "system", "content": "translate this sentence in docker command"},
    {"role": "user", "content": "Please show me the Docker containers that have exited and are related to the mongo image."},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1]['content'])

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


docker ps --filter "status=exited" --filter "image=mongo"


In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=base_model,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    tokenizer = tokenizer
)
messages = [
    {"role": "system", "content": "translate this sentence in docker command"},
    {"role": "user", "content": "Please show me the Docker containers that have exited and are related to the mongo image."},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
    temperature = 0.5,
    top_p  = 0.9,
    top_k = 10,
    do_sample = True,
    repetition_penalty = 1.2


)
print(outputs[0]["generated_text"][-1]['content'])

Device set to use cuda:0


docker ps --filter "status=exited" -a mongo
